In [1]:
import pandas as pd
import datetime
from tqdm import tqdm
from p_tqdm import p_map
from functools import partial

In [2]:
df = pd.read_csv('./data/order_brush_order.csv')
df.event_time = pd.to_datetime(df.event_time)

In [3]:
df = df.sort_values(by=['shopid','event_time'])
df 

,orderid,shopid,userid,event_time
59,31086409141107,10009,196962305,2019-12-27 03:06:50
76187,31144571933461,10051,2854032,2019-12-27 19:16:11
9055,31254979546679,10051,48600461,2019-12-29 01:56:19
188513,31195675919209,10061,168750452,2019-12-28 09:27:55
174620,31205132327893,10061,194819216,2019-12-28 12:05:32
...,...,...,...,...
81950,31493203163305,214949521,46269178,2019-12-31 20:06:43
108903,31328775676314,214964814,200983383,2019-12-29 22:26:16
203031,31453591917585,215175775,13688804,2019-12-31 09:06:31
42842,31472077221692,215175775,129266028,2019-12-31 14:14:37


# method

In [19]:
def check(_shopid,_df=df):
    import datetime
    sub_df = _df[_df.shopid==_shopid]
    if len(sub_df)>2:
        tar_userid = []
        for _time in sub_df.event_time:
            _target = sub_df[(sub_df.event_time>=_time) &
                             (sub_df.event_time-_time<=datetime.timedelta(0,3600,0))
                            ]
            num_order1 = len(_target)
            num_user1  = len(set(_target.userid))     
            if (num_order1/num_user1)>=3: 
                most_count = min(max(_target.userid.value_counts()),3)
                _tdf = _target.userid.value_counts().reset_index()
                tar_userid.extend(_tdf[_tdf.userid>=most_count]['index'])

        tar_userid = list(set(tar_userid))
        return '&'.join([str(x) for x in sorted(tar_userid)]) if tar_userid else '0'
    else:
        return '0'

check_p = partial(check,_df=df)

In [20]:
res111 = p_map(check_p,list(df.shopid.unique()),num_cpus=14)

In [37]:
output = pd.DataFrame({
    'shopid':df.shopid.unique(),
    'userid':res111
})

In [22]:
output[output.userid!='0']

,shopid,userid
40,10402,77819
57,10536,672345
111,42472,740844
114,42818,170385453
129,76934,190449497
...,...,...
17401,203531250,114282846
17960,204225676,198662175
18155,208696908,214111334
18557,210197928,52867898


In [23]:
output.to_csv('result_6.csv',index=False)

# baseline method

In [14]:
AA = df.groupby(['shopid',
                 'userid']).agg({'event_time': ['first','last', 'count']}).reset_index()
AA

shopid     userid          event_time                          
                                           first                last count
0           10009  196962305 2019-12-27 03:06:50 2019-12-27 03:06:50     1
1           10051    2854032 2019-12-27 19:16:11 2019-12-27 19:16:11     1
2           10051   48600461 2019-12-29 01:56:19 2019-12-29 01:56:19     1
3           10061   62464559 2019-12-31 02:58:48 2019-12-31 02:58:48     1
4           10061  130633421 2019-12-30 21:35:15 2019-12-30 21:35:15     1
...           ...        ...                 ...                 ...   ...
204998  214949521   46269178 2019-12-31 19:46:37 2019-12-31 20:06:43     2
204999  214964814  200983383 2019-12-29 22:26:16 2019-12-29 22:26:16     1
205000  215175775   13688804 2019-12-31 09:06:31 2019-12-31 09:06:31     1
205001  215175775  129266028 2019-12-31 14:14:37 2019-12-31 14:14:37     1
205002  215435223   79515106 2019-12-31 18:34:56 2019-12-31 18:34:56     1

[205003 rows x 5 columns]

In [15]:
AA['time_diff'] = AA[('event_time', 'last')] - AA[('event_time', 'first')] 

In [16]:
AA.columns

MultiIndex([(    'shopid',      ''),
            (    'userid',      ''),
            ('event_time', 'first'),
            ('event_time',  'last'),
            ('event_time', 'count'),
            ( 'time_diff',      '')],
           )

In [17]:
result_tmp = AA[(AA['time_diff']<datetime.timedelta(0,3600,0)) & (AA[('event_time', 'count')]>2)]

In [18]:
result_tmp

shopid     userid          event_time                            \
                                           first                last count   
426         10159  214988798 2019-12-29 20:38:09 2019-12-29 20:56:43     3   
901         10287  198097381 2019-12-30 00:17:17 2019-12-30 00:22:05     3   
907         10287  214226569 2019-12-31 01:13:02 2019-12-31 01:26:46     3   
1108        10402      77819 2019-12-28 14:24:33 2019-12-28 14:30:26     3   
1847        29583  214495220 2019-12-27 23:12:14 2019-12-27 23:27:59     4   
...           ...        ...                 ...                 ...   ...   
202133  203070501  189834273 2019-12-31 09:05:38 2019-12-31 09:08:54     3   
202410  203440274   73308605 2019-12-30 18:31:20 2019-12-30 18:50:58     3   
203527  204225676  198662175 2019-12-31 00:11:52 2019-12-31 00:19:37     5   
204646  210197928   52867898 2019-12-27 07:47:47 2019-12-27 07:49:56     4   
204901  213141071  213646699 2019-12-27 01:16:42 2019-12-27 01:42:23     3   

       time_diff  
                  
426     00:18:34  
901     00:04:48  
907     00:13:44  
1108    00:05:53  
1847    00:15:45  
...          ...  
202133  00:03:16  
202410  00:19:38  
203527  00:07:45  
204646  00:02:09  
204901  00:25:41  

[563 rows x 6 columns]

In [12]:
def idjoin(array):
    return "&".join( [str(x) for x in sorted(array)])

In [34]:
result = result_tmp.groupby('shopid').agg({'userid':idjoin}).reset_index()
result.shopid = result.shopid.astype('str')

In [35]:
result

,shopid,userid
0,10159,214988798
1,10287,198097381&214226569
2,10402,77819
3,29583,214495220&215275495
4,42472,740844
...,...,...
336,203070501,189834273
337,203440274,73308605
338,204225676,198662175
339,210197928,52867898


In [36]:
output = pd.DataFrame({
    'shopid':df.shopid.unique().astype('str'),
})

In [40]:
output = output.merge(result,how='left',on='shopid').fillna('0')
output

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0
...,...,...
18765,214662358,0
18766,214949521,0
18767,214964814,0
18768,215175775,0


In [43]:
output.to_csv('test1.csv',index=False)

# Check

In [34]:
df[(df.shopid==769445) & (df.userid==141006168)]

,orderid,shopid,userid,event_time
154185,31456249828907,769445,141006168,2019-12-31 09:50:50
49050,31456354719867,769445,141006168,2019-12-31 09:52:34
60958,31456703358525,769445,141006168,2019-12-31 09:58:23


In [36]:
df[(df.shopid==769445) & (df.event_time>datetime.datetime(2019,12,31,9,50,49))]

,orderid,shopid,userid,event_time
154185,31456249828907,769445,141006168,2019-12-31 09:50:50
49050,31456354719867,769445,141006168,2019-12-31 09:52:34
60958,31456703358525,769445,141006168,2019-12-31 09:58:23
99918,31459510711963,769445,155766871,2019-12-31 10:45:10
10172,31459574221430,769445,215368881,2019-12-31 10:46:14
26864,31473426025428,769445,1120425,2019-12-31 14:37:06
26285,31479694321078,769445,29299481,2019-12-31 16:21:34
185135,31481113985637,769445,45602398,2019-12-31 16:45:13
57893,31481538359554,769445,215264704,2019-12-31 16:52:18
37225,31484305653032,769445,74439423,2019-12-31 17:38:26
